# Basic buildling blocks

<div class="alert alert-block alert-warning">
Replace <code>YOUR_GITHUB_TOKEN</code> in the install script. To get your token follow the instructions in the <a href="../README.md">README.md</a>
</div>

In [ ]:
%pip install  "https://us-central1-data-359211.cloudfunctions.net/github-proxy/superlinked-0.0.9-py3-none-any.whl?token=YOUR_GITHUB_TOKEN"

Import the dependencies.

In [ ]:
from framework.common.schema.schema import String, schema
from framework.dsl.executor.in_memory.in_memory_executor import InMemoryExecutor
from framework.dsl.index.index import Index
from framework.dsl.query.param import Param
from framework.dsl.query.query import Query
from framework.dsl.source.in_memory_source import InMemorySource
from framework.dsl.space.text_similarity_space import TextSimilaritySpace

Create a schema for your data.

In [ ]:
@schema
class ParagraphSchema:
    body: String

Instatiate a new instance of your schema to start the pipeline definition.

In [ ]:

paragraph = ParagraphSchema()

Create a space that will run a transformers model on the body of the paragraph.

In [ ]:
relevance_space = TextSimilaritySpace(text=paragraph.body, model="all-MiniLM-L6-v2")

Group your space in an index to make it retrievable.

In [ ]:
paragraph_index = Index(relevance_space)

Define a query that will search for similar paragraphs in the index. The parameters will be filled later on.

In [ ]:
query = (
    Query(paragraph_index)
    .find(paragraph)
    .similar(relevance_space.text, Param("query_text"))
)

Create an in-memory source and executor to try out your configuration

In [ ]:
source: InMemorySource = InMemorySource(paragraph)
executor = InMemoryExecutor(sources=[source], indices=[paragraph_index])
app = executor.run()

Insert some example data.

In [ ]:
source.put([{"id": "happy_dog", "body": "That is a happy dog"}])
source.put([{"id": "happy_person", "body": "That is a very happy person"}])
source.put([{"id": "sunny_day", "body": "Today is a sunny day"}])

Query you data.

In [ ]:
result = app.query(query, query_text="This is a happy person")

for entity in result.entities:
    print(entity.id_.object_id)

Check how a different query can produce different results.

In [ ]:
result = app.query(query, query_text="This is a happy dog")

for entity in result.entities:
    print(entity.id_.object_id)